## H2O Model Playground

In [14]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

file_path = "../data/mini_all_features.csv"

In [15]:
from pyspark.sql import SparkSession
from pysparkling import *
import h2o



# Create Spark session
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

h2o.init()
# Initialize H2OContext
hc = H2OContext.getOrCreate()

# Get H2OConf from H2OContext
h2o_conf = hc.getConf()

# Set H2OConf properties
h2o_conf.set("spark.ext.h2o.client.language", "python")


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,4 hours 15 mins
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 1 day
H2O_cluster_name:,sparkling-water-jacobfletcher_local-1705869136820
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,821 Mb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Sparkling Water configuration:
  backend cluster mode : internal
  workers              : None
  cloudName            : Not set yet, it will be set automatically before starting H2OContext.
  base port            : 54321
  cloudTimeout         : 60000
  log level            : INFO
  nthreads             : -1
  drddMulFactor        : 10


In [16]:
h2o_df = h2o.import_file(file_path)
# h2o_df = spark.read.option("header", True).csv(file_path)
h2o_df = h2o_df.drop("userId")

try:
    h2o_df.drop('C1')
    h2o_df.drop('userId')
except Exception as e:
    pass

col_features = h2o_df.columns
col_features.remove("label")
# Split the data into training and validation sets
# Split the data into training and validation sets
train, valid = h2o_df.split_frame(ratios=[0.85])

# Define predictor and response columns
predictor_cols = col_features
response_col = "label"




Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [17]:


# Define hyperparameter grid
hyperparams = {
    'alpha': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0],  # Regularization parameter
    'lambda': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0],
}

# Initialize the linear regression model
linear_reg = H2OGeneralizedLinearEstimator(family='binomial', link="logit")

# Perform grid search
grid_search = H2OGridSearch(linear_reg, hyperparams)
grid_search.train(x=predictor_cols, y=response_col, training_frame=train)

# Get the best model from the grid search
best_model = grid_search.get_grid()[0]

# Make predictions on the validation set
preds = best_model.predict(valid)

# Get model performance on the validation set
validation_performance = best_model.model_performance(valid)


glm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


Adding alpha array to hyperparameter runs slower with gridsearch. This is due to the fact that the algo has to run initialization for every alpha value. Setting the alpha array as a model parameter will skip the initialization and run faster overall.


glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [18]:

print(validation_performance.F1())
print(validation_performance.F2())

[[0.24181351431681494, 0.8333333333333334]]
[[0.09730631198576863, 0.8593749999999999]]


In [19]:
validation_performance

ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.16951986390794138
RMSE: 0.4117279003273174
LogLoss: 0.7855791403044211
AUC: 0.8412698412698413
AUCPR: 0.6923471862733975
Gini: 0.6825396825396826
Null degrees of freedom: 32
Residual degrees of freedom: -2
Null deviance: 47.3911862061163
Residual deviance: 51.84822326009173
AIC: 121.84822326009173

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.24181351431681494
       0    1    Error    Rate
-----  ---  ---  -------  ----------
0      19   2    0.0952   (2.0/21.0)
1      2    10   0.1667   (2.0/12.0)
Total  21   12   0.1212   (4.0/33.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.241814     0.833333  11
max f2                       0.0973063    0.859375  15
max f0point5                 0.241814     0.833333  11
max accuracy                 0.241814     0.878788  11
max precision                0.999958     1         0
max recall                   0.00070399   1         28
max specificity              0.999958     1         0
max absolute_mcc             0.241814     0.738095  11
max min_per_class_accuracy   0.241814     0.833333  11
max mean_per_class_accuracy  0.241814     0.869048  11
max tns                      0.999958     21        0
max fns                      0.999958     11        0
max fps                      1.48529e-05  21        32
max tps                      0.00070399   12        28
max tnr                      0.999958     1         0
max fnr                      0.999958     0.916667  0
max fpr                      1.48529e-05  1         32
max tpr                      0.00070399   1         28

Gains/Lift Table: Avg response rate: 36.36 %, avg score: 25.76 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.030303                    0.999864           2.75      2.75               1                0.999958     1                           0.999958            0.0833333       0.0833333                  175       175                0.0833333
2        0.030303                    0.999769           0         2.75               0                0            1                           0.999958            0               0.0833333                  -100      175                0.0833333
3        0.030303                    0.999674           0         2.75               0                0            1                           0.999958            0               0.0833333                  -100      175                0.0833333
4        0.0606061                   0.929517           0         1.375              0                0.999663     0.5                         0.99981             0               0.0833333                  -100      37.5               0.0357143
5        0.0606061                   0.84935            0         1.375              0                0            0.5                         0.99981             0               0.0833333                  -100      37.5               0.0357143
6        0.121212                    0.688059           1.375     1.375              0.5              0.718705     0.5                         0.859258            0.0833333       0.166667                   37.5      37.5               0.0714286
7        0.151515                    0.66186            2.75      1.65               1                0.687214     0.6                         0.824849            0.0833333 

## H2O Gradient Boosting

In [20]:



h2o_df = h2o.import_file(file_path)
# h2o_df = spark.read.option("header", True).csv(file_path)
h2o_df = h2o_df.drop("userId")

try:
    h2o_df.drop('C1')
    h2o_df.drop('userId')
except Exception as e:
    pass

col_features = h2o_df.columns
col_features.remove("label")
# Split the data into training and validation sets
# Split the data into training and validation sets
train, valid = h2o_df.split_frame(ratios=[0.85])

# Define predictor and response columns
predictor_cols = col_features
response_col = "label"


# Define hyperparameter grid
hyperparams = {
    'learn_rate': [0.01, 0.1, 0.2],
    'ntrees': [50, 100, 200],
    'max_depth': [3, 4, 5],
}

gradient_boost = H2OGradientBoostingEstimator(seed=42)


# Perform grid search
grid_search = H2OGridSearch(gradient_boost, hyperparams)
grid_search.train(x=predictor_cols, y=response_col, training_frame=train)

# Get the best model from the grid search
best_model = grid_search.get_grid()[0]

# Make predictions on the validation set
preds = best_model.predict(valid)

# Get model performance on the validation set
validation_performance = best_model.model_performance(valid)



Parse progress: |

████████████████████████████████████████████████████████████████| (done) 100%
gbm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [ ]:

validation_performance

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 0.15326187011957534
RMSE: 0.39148674322328636
MAE: 0.29938115856573133
RMSLE: 0.27569242700388985
Mean Residual Deviance: 0.15326187011957534

## Distributed Random Forrest

In [27]:
from h2o.estimators import H2ORandomForestEstimator
h2o_df = h2o.import_file(file_path)
# h2o_df = spark.read.option("header", True).csv(file_path)
h2o_df = h2o_df.drop("userId")

try:
    h2o_df.drop('C1')
    h2o_df.drop('userId')
except Exception as e:
    pass

col_features = h2o_df.columns
col_features.remove("label")
# Split the data into training and validation sets
# Split the data into training and validation sets
train, valid = h2o_df.split_frame(ratios=[0.85])

# Define predictor and response columns
predictor_cols = col_features
response_col = "label"


# Define hyperparameter grid
hyperparams = {
    'ntrees': [50, 100, 150],
    'max_depth': [5, 10, 15],
    'min_rows': [1, 5, 10],
    'sample_rate': [0.7, 0.8, 0.9],
    'col_sample_rate_per_tree': [0.7, 0.8, 0.9],
    'min_split_improvement': [1e-4, 1e-3, 1e-2]
}

drf_estimator = H2ORandomForestEstimator(
    calibrate_model=False,
    calibration_frame=valid,
    binomial_double_trees=True,
    nfolds=5,  # Specify the number of folds for cross-validation
    fold_assignment='Stratified',  # Use stratified cross-validation for binomial classification
    keep_cross_validation_predictions=True  # Keep predictions on the validation folds
)



grid_search = H2OGridSearch(drf_estimator, hyperparams)
grid_search.train(x=predictor_cols, y=response_col, training_frame=train)

best_model = grid_search.get_grid()[0]

preds = best_model.predict(valid)

validation_performance = best_model.model_performance(valid)



# predictors = [col for col in train_data.columns if col.endswith("_scaled")]
# response = "label"

# model_drf = estimator.train(x=predictors,
#                y=response,
#                training_frame=train_scaled_data,
#                validation_frame=test_scaled_data)



# # Eval performance:
# perf = model_drf.model_performance()

# # Generate predictions on a validation set (if necessary):
# pred = model_drf.predict(test_scaled_data)

Parse progress: |

████████████████████████████████████████████████████████████████| (done) 100%
drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
Errors/Warnings building gridsearch model

Hyper-parameter: col_sample_rate_per_tree, 0.8
Hyper-parameter: max_depth, 15
Hyper-parameter: min_rows, 1.0
Hyper-parameter: min_split_improvement, 0.001
Hyper-parameter: ntrees, 150
Hyper-parameter: sample_rate, 0.7
failure_details: Illegal argument(s) for DRF model: Grid_DRF_py_35_sid_8c18_model_python_1705869686358_60_model_197_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory ( 323  B per tree x 150 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: Grid_DRF_py_35_sid_8c18_model_python_1705869686358_60_model_197_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the 

In [33]:

validation_performance

ModelMetricsRegression: drf
** Reported on test data. **

MSE: 0.10115762044157485
RMSE: 0.31805285793649907
MAE: 0.2617688206436508
RMSLE: 0.22971050148870115
Mean Residual Deviance: 0.10115762044157485

In [34]:
h2o.cluster().shutdown()

H2O session _sid_8c18 closed.
